In [1]:
#upload zipped folder to kaggle. The folde contains different subfolders whose folder name are the labels
#base folder
dataset_path = '/kaggle/input/tf-flowers/flower_photos'

In [2]:
# List the sub folders whose name represents labels
import os

for folder_name in os.listdir(dataset_path):
    print(folder_name)

LICENSE.txt
dandelion
sunflowers
daisy
roses
tulips


In [5]:
# Create lists to hold image paths and labels
image_paths = []
labels = []

# Iterate through each folder in the dataset
for label in os.listdir(dataset_path):
    label_path = os.path.join(dataset_path, label)
    
    if os.path.isdir(label_path):  # Make sure it's a directory
        for file_name in os.listdir(label_path):
            if file_name.endswith(('jpg', 'png', 'jpeg')):  # Assuming you're working with these formats
                image_paths.append(os.path.join(label_path, file_name))
                labels.append(label)

In [7]:
class_names = list(set(labels))
class_names

['tulips', 'daisy', 'dandelion', 'roses', 'sunflowers']

In [ ]:
#===================== START FROM HERE ============================
#if dataset already uploaded to kaggle

In [ ]:
#remove existing folder if present
import shutil
shutil.rmtree("/kaggle/working/split_dataset")

In [10]:
#Create output directory to create folders for train, validation and test

import os
import shutil
import random

# Define the source and destination directories
source_dir ='/kaggle/input/tf-flowers/flower_photos'
split_dir = 'split_dataset'

# Create directories for train, validation, and test
os.makedirs(os.path.join(split_dir, 'train'), exist_ok=True)
os.makedirs(os.path.join(split_dir, 'validation'), exist_ok=True)
os.makedirs(os.path.join(split_dir, 'test'), exist_ok=True)

In [11]:
#copy image files to new folders

# Iterate over each class folder
for class_folder in os.listdir(source_dir):
    class_path = os.path.join(source_dir, class_folder)
    
    if os.path.isdir(class_path):  # Check if it's a folder
        # Create class directories in split directory
        os.makedirs(os.path.join(split_dir, 'train', class_folder), exist_ok=True)
        os.makedirs(os.path.join(split_dir, 'validation', class_folder), exist_ok=True)
        os.makedirs(os.path.join(split_dir, 'test', class_folder), exist_ok=True)

        # List all images in the class folder
        all_images = os.listdir(class_path)
        
        # Shuffle images for randomness
        random.shuffle(all_images)

        # Split images
        total_images = len(all_images)
        train_count = int(0.7 * total_images)  # 70% for training
        valid_count = int(0.2 * total_images)  # 20% for validation
        test_count = total_images - train_count - valid_count  # Rest for testing

        # Move images to respective folders
        for img in all_images[:train_count]:
            shutil.copy(os.path.join(class_path, img), os.path.join(split_dir, 'train', class_folder, img))
        
        for img in all_images[train_count:train_count + valid_count]:
            shutil.copy(os.path.join(class_path, img), os.path.join(split_dir, 'validation', class_folder, img))

        for img in all_images[train_count + valid_count:]:
            shutil.copy(os.path.join(class_path, img), os.path.join(split_dir, 'test', class_folder, img))


In [16]:
import numpy as np
class_names = list(np.sort(class_names))
class_names
#class_names = ['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips' ]

['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']

In [20]:
n_classes = len(class_names)
n_classes

5

In [21]:
print("Training: ", train_count)
print("Validation: ", valid_count)
print("Testing: ", test_count)

Training:  559
Validation:  159
Testing:  81


In [22]:
# Load datasets
from tensorflow import keras

train_dir = '/kaggle/working/split_dataset/train'  
valid_dir = '/kaggle/working/split_dataset/validation' 
test_dir = '/kaggle/working/split_dataset/test'

batch_size = 32
img_height = 224
img_width = 224

train_set_raw = keras.preprocessing.image_dataset_from_directory(
    train_dir,
    batch_size = batch_size,
    image_size = (img_height, img_width),
    shuffle=True
)

valid_set_raw = keras.preprocessing.image_dataset_from_directory(
    valid_dir,
    batch_size = batch_size,
    image_size = (img_height, img_width),
    shuffle=True
)

test_set_raw = keras.preprocessing.image_dataset_from_directory(
    test_dir,
    batch_size = batch_size,
    image_size = (img_height, img_width),
    shuffle=True
)

2025-07-16 11:00:18.450750: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752663618.639457      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752663618.696291      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Found 2567 files belonging to 5 classes.


I0000 00:00:1752663631.419010      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


Found 731 files belonging to 5 classes.
Found 372 files belonging to 5 classes.


In [23]:
train_set_raw

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [24]:
test_set_raw

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [25]:
#preprocess the images appropriately for the Xception model
preprocess = keras.layers.Lambda(keras.applications.xception.preprocess_input)

In [26]:
#Get processed datasets
train_set = train_set_raw.map(lambda X, y: (preprocess(X), y))
train_set = train_set.shuffle(1000, seed=42).prefetch(1)
valid_set = valid_set_raw.map(lambda X, y: (preprocess(X), y))
test_set = test_set_raw.map(lambda X, y: (preprocess(X), y))

In [27]:
train_set

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [28]:
valid_set

<_MapDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [29]:
#Data augmentation to increase number of instances
data_augmentation = keras.Sequential([
    keras.layers.RandomFlip(mode="horizontal", seed=42),
    keras.layers.RandomRotation(factor=0.05, seed=42),
    keras.layers.RandomContrast(factor=0.2, seed=42)
])

In [ ]:
#load an Xception model, pretrained on ImageNet
base_model = keras.applications.xception.Xception(
    weights="imagenet", 
    include_top=False)

In [ ]:
# print base model layers
for (i,layer) in enumerate(base_model.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)

In [ ]:
#global average pooling layer
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)

#dense output layer
output = keras.layers.Dense(n_classes, activation="softmax")(avg)

In [ ]:
#Build Keras model
model = keras.Model(inputs=base_model.input, outputs=output)

In [ ]:
#freeze the weights of the pretrained layers, for 3 epochs at beginning of training
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
#compile model
optimizer = keras.optimizers.SGD(learning_rate=0.1, momentum=0.9)

model.compile(
    loss="sparse_categorical_crossentropy", 
    optimizer=optimizer, 
    metrics=["accuracy"])

In [ ]:
print(model.summary())

In [ ]:
#train model
history = model.fit(train_set, validation_data=valid_set, epochs=3)

In [ ]:
#unfreeze layers 56 and above of pretrained model
for layer in base_model.layers[56:]:
    layer.trainable = True

In [ ]:
#compile the model
optimizer = keras.optimizers.SGD(learning_rate=0.01, momentum=0.9)

model.compile(
    loss="sparse_categorical_crossentropy", 
    optimizer=optimizer, 
    metrics=["accuracy"])

In [ ]:
print(model.summary())

In [ ]:
#train final model
history = model.fit(train_set, validation_data=valid_set, epochs=10)

In [ ]:
# Plot the loss and accuracy curves for training and validation 
from matplotlib import pyplot as plt

plt.plot(history.history['val_loss'], color='b', label="validation loss")
plt.title("Test Loss")
plt.xlabel("Number of Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
model.evaluate(test_set)

In [ ]:
prediction = model.predict(test_set)

In [ ]:
#get actual categories
import tensorflow as tf
true_categories = tf.concat([y for x, y in test_set], axis=0)

#get predicted categories
predicted_categories = tf.argmax(prediction, axis=1)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

print('Confusion Matrix')
confusion_matrix(predicted_categories, true_categories)

In [ ]:
#Testing of a folder
import numpy as np
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import load_img

directory_path = '/kaggle/working/split_dataset/test/dandelion'
values = []
for root, _, files in os.walk(directory_path):
        for file in files:
            full_file_path = os.path.join(root, file)
            #print(full_file_path)
            test_image = load_img(full_file_path,target_size = (img_height, img_width, 3)) 
            test_image = image.img_to_array(test_image)
            test_image = np.expand_dims(test_image,axis = 0)
            test_image = keras.applications.xception.preprocess_input(test_image)
            result = np.argmax(model.predict(test_image))
            values.append(result)

for i in range(5):
    print(class_names[i], values.count(i))